In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display, Markdown, Latex
import dataset.labelEncoder

sns.set_style('whitegrid')

from sklearn.preprocessing import LabelEncoder
from sklearn import model_selection
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import f1_score


# 1加载预览数据集
Loading and Understanding the dataset


In [2]:
df_loan = pd.read_csv("/Users/mxfmeng/PycharmProjects/GraduationDesign/dataset/loan_old.csv")
df_loan.head(7)

/var/folders/8m/6cpmqrns19n25q6dpm3wnyzh0000gn/T/ipykernel_50311/2773092102.py:1: DtypeWarning: Columns (19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  df_loan = pd.read_csv("/Users/mxfmeng/PycharmProjects/GraduationDesign/dataset/loan_old.csv")


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,1077501,1296599,5000.0,5000.0,4975.0,36 months,10.65,162.87,B,B2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1077430,1314167,2500.0,2500.0,2500.0,60 months,15.27,59.83,C,C4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1077175,1313524,2400.0,2400.0,2400.0,36 months,15.96,84.33,C,C5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1076863,1277178,10000.0,10000.0,10000.0,36 months,13.49,339.31,C,C1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1075358,1311748,3000.0,3000.0,3000.0,60 months,12.69,67.79,B,B5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1075269,1311441,5000.0,5000.0,5000.0,36 months,7.90,156.46,A,A4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1069639,1304742,7000.0,7000.0,7000.0,60 months,15.96,170.08,C,C5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Description of dataset

In [3]:
df_loan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 887379 entries, 0 to 887378
Data columns (total 74 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   id                           887379 non-null  int64  
 1   member_id                    887379 non-null  int64  
 2   loan_amnt                    887379 non-null  float64
 3   funded_amnt                  887379 non-null  float64
 4   funded_amnt_inv              887379 non-null  float64
 5   term                         887379 non-null  object 
 6   int_rate                     887379 non-null  float64
 7   installment                  887379 non-null  float64
 8   grade                        887379 non-null  object 
 9   sub_grade                    887379 non-null  object 
 10  emp_title                    835917 non-null  object 
 11  emp_length                   842554 non-null  object 
 12  home_ownership               887379 non-null  object 
 13 

# 2删除不相关列
Removing Irrelevant columns
新的数据集保留较为重要的11列，而删除其他列

In [4]:
savedColumns = ['loan_amnt','term','int_rate','installment','grade','emp_length','home_ownership','annual_inc','verification_status','loan_status','purpose',]
df_loan.drop(df_loan.columns.difference(savedColumns), axis=1, inplace=True)
df_loan.head(10)

,loan_amnt,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose
0,5000.0,36 months,10.65,162.87,B,10+ years,RENT,24000.0,Verified,Fully Paid,credit_card
1,2500.0,60 months,15.27,59.83,C,< 1 year,RENT,30000.0,Source Verified,Charged Off,car
2,2400.0,36 months,15.96,84.33,C,10+ years,RENT,12252.0,Not Verified,Fully Paid,small_business
3,10000.0,36 months,13.49,339.31,C,10+ years,RENT,49200.0,Source Verified,Fully Paid,other
4,3000.0,60 months,12.69,67.79,B,1 year,RENT,80000.0,Source Verified,Current,other
5,5000.0,36 months,7.90,156.46,A,3 years,RENT,36000.0,Source Verified,Fully Paid,wedding
6,7000.0,60 months,15.96,170.08,C,8 years,RENT,47004.0,Not Verified,Current,debt_consolidation
7,3000.0,36 months,18.64,109.43,E,9 years,RENT,48000.0,Source Verified,Fully Paid,car
8,5600.0,60 months,21.28,152.39,F,4 years,OWN,40000.0,Source Verified,Charged Off,small_business
9,5375.0,60 months,12.69,121.45,B,< 1 year,RENT,15000.0,Verified,Charged Off,other


In [5]:
df_loan.isnull().sum()

loan_amnt                  0
term                       0
int_rate                   0
installment                0
grade                      0
emp_length             44825
home_ownership             0
annual_inc                 4
verification_status        0
loan_status                0
purpose                    0
dtype: int64

In [6]:
df_loan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 887379 entries, 0 to 887378
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   loan_amnt            887379 non-null  float64
 1   term                 887379 non-null  object 
 2   int_rate             887379 non-null  float64
 3   installment          887379 non-null  float64
 4   grade                887379 non-null  object 
 5   emp_length           842554 non-null  object 
 6   home_ownership       887379 non-null  object 
 7   annual_inc           887375 non-null  float64
 8   verification_status  887379 non-null  object 
 9   loan_status          887379 non-null  object 
 10  purpose              887379 non-null  object 
dtypes: float64(4), object(7)
memory usage: 74.5+ MB


annual_inc列（年收入）缺失数据填充为0，emp_length列（工作年限）缺失数据暂时不处理（未来使用该数据集时可能删除缺失的数据）

In [7]:
df_loan.annual_inc = df_loan.annual_inc.fillna(0)
df_loan.dropna(subset=['emp_length'], inplace=True)
df_loan.info()

<class 'pandas.core.frame.DataFrame'>
Index: 842554 entries, 0 to 887378
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   loan_amnt            842554 non-null  float64
 1   term                 842554 non-null  object 
 2   int_rate             842554 non-null  float64
 3   installment          842554 non-null  float64
 4   grade                842554 non-null  object 
 5   emp_length           842554 non-null  object 
 6   home_ownership       842554 non-null  object 
 7   annual_inc           842554 non-null  float64
 8   verification_status  842554 non-null  object 
 9   loan_status          842554 non-null  object 
 10  purpose              842554 non-null  object 
dtypes: float64(4), object(7)
memory usage: 77.1+ MB


# 3创建标签列，描述贷款状态
将标签列loan_status（贷款状态）二元分类
0：低风险
1：高风险

In [8]:
#统计分类前所有status出现的次数
counts = df_loan['loan_status'].value_counts()
print("BEFORE binary classification:\n",counts)
# binary classification
label_categories = [
    (0, ['Fully Paid', 'Does not meet the credit policy. Status:Fully Paid', 'Current']),
    (1, ['Late (31-120 days)', 'Late (16-30 days)', 'In Grace Period',
         'Charged Off', 'Default', 'Does not meet the credit policy. Status:Charged Off'])
]

# function to apply the transformation
def classify_label(text):
    for category, matches in label_categories:
        if any(match in text for match in matches):
            return category
    return None

df_loan.loc[:, 'label'] = df_loan['loan_status'].apply(classify_label)
df_loan = df_loan.drop('loan_status', axis=1)

#统计分类后新标签label的出现次数
counts = df_loan['label'].value_counts()
print("AFTER binary classification:\n",counts)

BEFORE binary classification:
 loan_status
Current                                                568686
Fully Paid                                             200351
Charged Off                                             42723
Late (31-120 days)                                      10872
Issued                                                   7829
In Grace Period                                          6015
Late (16-30 days)                                        2215
Does not meet the credit policy. Status:Fully Paid       1969
Default                                                  1148
Does not meet the credit policy. Status:Charged Off       746
Name: count, dtype: int64
AFTER binary classification:
 label
0.0    771006
1.0     63719
Name: count, dtype: int64


将某些标称属性和序数属性，统统转为纯数字的表示形式

In [9]:
def SC_LabelEncoder1(text):
    if text == "E":
        return 1
    elif text == "D":
        return 2
    elif text == "C":
        return 3
    elif text == "B":
        return 4
    elif text == "A":
        return 5
    else:
        return 0


def SC_LabelEncoder2(text):
    if text == "< 1 year":
        return 1
    elif text == "1 year":
        return 2
    elif text == "2 years":
        return 3
    elif text == "3 years":
        return 4
    elif text == "4 years":
        return 5
    elif text == "5 years":
        return 6
    elif text == "6 years":
        return 7
    elif text == "7 years":
        return 8
    elif text == "8 years":
        return 9
    elif text == "9 years":
        return 10
    elif text == "10 years":
        return 11
    elif text == "10+ years":
        return 12
    else:
        return 0

def SC_LabelEncoder3(text):
    if text == "RENT":
        return 1
    elif text == "MORTGAGE":
        return 2
    elif text == "OWN":
        return 3
    else:
        return 0

df_loan["grade"] = df_loan["grade"].apply(SC_LabelEncoder1)
df_loan["emp_length"] = df_loan["emp_length"].apply(SC_LabelEncoder2)
df_loan["home_ownership"] = df_loan["home_ownership"].apply(SC_LabelEncoder3)
df_loan.head(10)

,loan_amnt,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,purpose,label
0,5000.0,36 months,10.65,162.87,4,12,1,24000.0,Verified,credit_card,0.0
1,2500.0,60 months,15.27,59.83,3,1,1,30000.0,Source Verified,car,1.0
2,2400.0,36 months,15.96,84.33,3,12,1,12252.0,Not Verified,small_business,0.0
3,10000.0,36 months,13.49,339.31,3,12,1,49200.0,Source Verified,other,0.0
4,3000.0,60 months,12.69,67.79,4,2,1,80000.0,Source Verified,other,0.0
5,5000.0,36 months,7.90,156.46,5,4,1,36000.0,Source Verified,wedding,0.0
6,7000.0,60 months,15.96,170.08,3,9,1,47004.0,Not Verified,debt_consolidation,0.0
7,3000.0,36 months,18.64,109.43,1,10,1,48000.0,Source Verified,car,0.0
8,5600.0,60 months,21.28,152.39,0,5,3,40000.0,Source Verified,small_business,1.0
9,5375.0,60 months,12.69,121.45,4,1,1,15000.0,Verified,other,1.0


In [10]:
df_loan.info()

<class 'pandas.core.frame.DataFrame'>
Index: 842554 entries, 0 to 887378
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   loan_amnt            842554 non-null  float64
 1   term                 842554 non-null  object 
 2   int_rate             842554 non-null  float64
 3   installment          842554 non-null  float64
 4   grade                842554 non-null  int64  
 5   emp_length           842554 non-null  int64  
 6   home_ownership       842554 non-null  int64  
 7   annual_inc           842554 non-null  float64
 8   verification_status  842554 non-null  object 
 9   purpose              842554 non-null  object 
 10  label                834725 non-null  float64
dtypes: float64(5), int64(3), object(3)
memory usage: 77.1+ MB


# 4离散性数据处理
verification_status(认证状态), purpose(贷款目的),term(贷款期数)

In [11]:
# use LabelEncoder() to encode another category column:
for col in ["verification_status", "purpose","term"]:
    le = LabelEncoder()
    le.fit(df_loan[col])
    df_loan[col] = le.transform(df_loan[col])
df_loan.head()

,loan_amnt,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,purpose,label
0,5000.0,0,10.65,162.87,4,12,1,24000.0,2,1,0.0
1,2500.0,1,15.27,59.83,3,1,1,30000.0,1,0,1.0
2,2400.0,0,15.96,84.33,3,12,1,12252.0,0,11,0.0
3,10000.0,0,13.49,339.31,3,12,1,49200.0,1,9,0.0
4,3000.0,1,12.69,67.79,4,2,1,80000.0,1,9,0.0


In [18]:
df_loan.info()

<class 'pandas.core.frame.DataFrame'>
Index: 842554 entries, 0 to 887378
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   loan_amnt            842554 non-null  float64
 1   term                 842554 non-null  int64  
 2   int_rate             842554 non-null  float64
 3   installment          842554 non-null  float64
 4   grade                842554 non-null  int64  
 5   emp_length           842554 non-null  int64  
 6   home_ownership       842554 non-null  int64  
 7   annual_inc           842554 non-null  float64
 8   verification_status  842554 non-null  int64  
 9   purpose              842554 non-null  int64  
 10  label                834725 non-null  float64
dtypes: float64(5), int64(6)
memory usage: 77.1 MB


In [21]:
df_loan.dropna(subset=['label'], inplace=True)
df_loan.info()

<class 'pandas.core.frame.DataFrame'>
Index: 834725 entries, 0 to 887378
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   loan_amnt            834725 non-null  float64
 1   term                 834725 non-null  int64  
 2   int_rate             834725 non-null  float64
 3   installment          834725 non-null  float64
 4   grade                834725 non-null  int64  
 5   emp_length           834725 non-null  int64  
 6   home_ownership       834725 non-null  int64  
 7   annual_inc           834725 non-null  float64
 8   verification_status  834725 non-null  int64  
 9   purpose              834725 non-null  int64  
 10  label                834725 non-null  float64
dtypes: float64(5), int64(6)
memory usage: 76.4 MB


In [22]:
# 将处理后的数据集保存为新的csv文件
df_loan.to_csv('/Users/mxfmeng/PycharmProjects/GraduationDesign/dataset/loan_new.csv', index=False)